In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp
import sys

sys.path.insert(0, '/home/molly/Desktop/DeepTCGA/')
import load_data

In [6]:
def fc_layer(A_prev, size_in, size_out, name="fully-connected"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w) + b
        return act, w, b
    

def random_sparse_layer(A_prev, size_in, size_out, sparsity, name="sparse"):
    mask = generate_random_sparse_mask(size_in, size_out, sparsity)
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        w_masked = tf.multiply(mask, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w_masked) + b
        return act, w, w_masked, mask, b

    
def build_model(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2, b2 = fc_layer(hidden, N_HIDDEN, N_IN, name="fc1")
    parameters.update({"w2": w2, "b2": b2})
    return x_recon, parameters


def build_model_right_sparse(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1, b1 = fc_layer(x, N_IN, N_HIDDEN, name="fc1")
    parameters.update({"w1": w1, "b1": b1})    
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse1")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})
    return x_recon, parameters


def build_model_both_sparse_assymetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse2")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2}) 
    return x_recon, parameters


def build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    mask2 =tf.transpose(mask1)
    w2_init = tf.Variable(tf.truncated_normal([N_HIDDEN, N_IN], mean=0, stddev=0.1))
    w2_sparse = tf.multiply(mask2, w2_init)
    b2 = tf.Variable(tf.constant(0.1, shape=[N_IN]))
    x_recon = tf.matmul(hidden, w2_sparse) + b2
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})     
    return x_recon, parameters


def back_prop(x, x_recon, learning_rate):
    loss = tf.reduce_mean(tf.square(x_recon - x))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step


def feed_forward(x, parameters):
    w1_sparse, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1_sparse) + b1)
    w2, b2 = parameters["w2"], parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_right_sparse(x, parameters):
    w1, b1 = parameters["w1"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_both_sparse(x, parameters):
    w1, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden

def mse(x, x_recon, name=""):
    mse = tf.reduce_mean(tf.square(x_recon-x))
    mse_summary = tf.summary.scalar(name + "_mse", mse)
    return mse, mse_summary


def generate_random_sparse_mask(size_in, size_out, sparsity):
    """ make sure each node has at least one edge connecting to it"""
    assert(size_out>=100)
    while True:
        try:
            a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
            a = (a>(1-sparsity)).astype(int)
            assert((a.sum(axis=1)>1).all())
            assert((a.sum(axis=0)>1).all())
            break
        except AssertionError:
            continue
    return tf.constant(a.astype('float32'))

In [9]:
def train_model(data, batch_size=128, num_epoch=1000, learning_rate=1e-3, sparsity=1, mode="left", extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}".format(datetime.datetime.today().date())
    N_IN = data.train.num_features
    N_OUT = data.train.num_features
    N_HIDDEN = 1000
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_batch(batch_size=batch_size, 
                                                     count_by="epoch")
    x = train_batch["X"]
    if mode=="left":
        x_recon, parameters = build_model(x, N_IN, N_HIDDEN, sparsity=sparsity)
    elif mode == "both":
        x_recon, parameters = build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=sparsity)        
    else:
        raise
    train_step = back_prop(x, x_recon, learning_rate)    
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    if mode=="left":
        x_train_recon, train_latent = feed_forward(x_train, parameters)
        x_val_recon, val_latent = feed_forward(x_val, parameters)
    elif mode == "both":
        x_train_recon, train_latent = feed_forward_both_sparse(x_train, parameters)
        x_val_recon, val_latent = feed_forward_both_sparse(x_val, parameters)
    
    train_mse, train_summ = mse(x_train, x_train_recon, name="train")
    val_mse, val_summ = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "ae_{0}".format(extra))
    writer.add_graph(sess.graph)                            
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        sess.run([train_iter.initializer])
        t0 = time.time()
        try:
            while True:
                sess.run(train_step)
        except tf.errors.OutOfRangeError:
            sess.run([train_iter_all.initializer, val_iter.initializer])
            [train_error, train_s, val_error, val_s] = sess.run(
                [train_mse, train_summ, val_mse, val_summ])
            writer.add_summary(train_s, epoch)
            writer.add_summary(val_s, epoch)
            if epoch % 10 == 0:
                print("epoch", epoch)
                print("training mse:", train_error, "validation mse", val_error)
                print("epoch time:", time.time()-t0)
#     train_latent = sess.run(train_latent)
#     val_latent = sess.run(val_latent)
#     np.save("./results/AE/train_latent_sparse2.npy", train_latent)
#     np.save("./results/AE/val_latent_sparse2.npy", val_latent)    
    sess.close()

In [4]:
tcga = load_data.read_data_sets("../data/mRNA_lognorm_StandardScaled.hdf")

In [12]:
train_model(tcga, num_epoch=1000, sparsity=0.02, extra="1000_hidden_left_sparsity_0.02")

epoch 0
training mse: 2.0481305 validation mse 2.1306565
epoch time: 3.363515853881836
epoch 10
training mse: 0.33192283 validation mse 0.37197188
epoch time: 1.4703829288482666
epoch 20
training mse: 0.23760201 validation mse 0.27356526
epoch time: 1.497955560684204
epoch 30
training mse: 0.20179346 validation mse 0.24185064
epoch time: 1.512904167175293
epoch 40
training mse: 0.18144184 validation mse 0.22731383
epoch time: 1.494112491607666
epoch 50
training mse: 0.16849466 validation mse 0.21996418
epoch time: 1.4905786514282227
epoch 60
training mse: 0.1603287 validation mse 0.2166168
epoch time: 1.5001256465911865
epoch 70
training mse: 0.15563543 validation mse 0.21543533
epoch time: 1.5096299648284912
epoch 80
training mse: 0.15206401 validation mse 0.2143766
epoch time: 1.5092341899871826
epoch 90
training mse: 0.15033507 validation mse 0.21424161
epoch time: 1.5197970867156982
epoch 100
training mse: 0.14870311 validation mse 0.2136523
epoch time: 1.5090434551239014
epoch 110

epoch 900
training mse: 0.13675123 validation mse 0.2035255
epoch time: 1.5186290740966797
epoch 910
training mse: 0.13657768 validation mse 0.20340773
epoch time: 1.5246055126190186
epoch 920
training mse: 0.13681664 validation mse 0.20359358
epoch time: 1.5217702388763428
epoch 930
training mse: 0.13660383 validation mse 0.20341197
epoch time: 1.537276029586792
epoch 940
training mse: 0.13680685 validation mse 0.20359948
epoch time: 1.5159025192260742
epoch 950
training mse: 0.13652399 validation mse 0.20330776
epoch time: 1.5308334827423096
epoch 960
training mse: 0.1364057 validation mse 0.20321901
epoch time: 1.5223407745361328
epoch 970
training mse: 0.13681419 validation mse 0.20347553
epoch time: 1.5197114944458008
epoch 980
training mse: 0.13642982 validation mse 0.20307429
epoch time: 1.5268759727478027
epoch 990
training mse: 0.13634315 validation mse 0.20303975
epoch time: 1.540445327758789


In [11]:
for sparsity in [0.1, 0.02]:
    train_model(tcga, num_epoch=1000, mode="both", sparsity=sparsity,
                extra="1000_hidden_both_sparsity_{0}_symmetric".format(sparsity))

epoch 0
training mse: 1.4714371 validation mse 1.5129342
epoch time: 6.439133167266846
epoch 10
training mse: 0.36632422 validation mse 0.37878165
epoch time: 2.7417097091674805
epoch 20
training mse: 0.3212575 validation mse 0.33457178
epoch time: 1.6314873695373535
epoch 30
training mse: 0.30571523 validation mse 0.32063067
epoch time: 1.596449375152588
epoch 40
training mse: 0.2960907 validation mse 0.31247684
epoch time: 1.588893175125122
epoch 50
training mse: 0.28833872 validation mse 0.30640423
epoch time: 1.6200275421142578
epoch 60
training mse: 0.28212145 validation mse 0.3016359
epoch time: 1.6010632514953613
epoch 70
training mse: 0.2773217 validation mse 0.29731184
epoch time: 1.5988209247589111
epoch 80
training mse: 0.27331787 validation mse 0.29338077
epoch time: 1.6119561195373535
epoch 90
training mse: 0.26931396 validation mse 0.2887647
epoch time: 1.6160030364990234
epoch 100
training mse: 0.2650889 validation mse 0.28310993
epoch time: 1.5975489616394043
epoch 110


epoch 900
training mse: 0.23628344 validation mse 0.24818526
epoch time: 1.5522305965423584
epoch 910
training mse: 0.23624097 validation mse 0.2481841
epoch time: 1.5554273128509521
epoch 920
training mse: 0.23657301 validation mse 0.2485668
epoch time: 1.5656635761260986
epoch 930
training mse: 0.23708943 validation mse 0.2490237
epoch time: 1.5747883319854736
epoch 940
training mse: 0.2362689 validation mse 0.24821535
epoch time: 1.5540661811828613
epoch 950
training mse: 0.23643245 validation mse 0.24842495
epoch time: 1.5622467994689941
epoch 960
training mse: 0.23611586 validation mse 0.24799599
epoch time: 1.5762794017791748
epoch 970
training mse: 0.23668484 validation mse 0.24861097
epoch time: 1.5510225296020508
epoch 980
training mse: 0.23635665 validation mse 0.24834251
epoch time: 1.5515129566192627
epoch 990
training mse: 0.2368737 validation mse 0.2487706
epoch time: 1.5444788932800293
epoch 0
training mse: 0.8212296 validation mse 0.81481624
epoch time: 5.12799096107482

epoch 800
training mse: 0.4018245 validation mse 0.40160304
epoch time: 1.572314739227295
epoch 810
training mse: 0.40189207 validation mse 0.40161106
epoch time: 1.5624737739562988
epoch 820
training mse: 0.40188345 validation mse 0.40172103
epoch time: 1.5604610443115234
epoch 830
training mse: 0.40159783 validation mse 0.40142757
epoch time: 1.5666546821594238
epoch 840
training mse: 0.40173665 validation mse 0.40153068
epoch time: 1.5702307224273682
epoch 850
training mse: 0.4017203 validation mse 0.40148926
epoch time: 1.5685484409332275
epoch 860
training mse: 0.40163103 validation mse 0.40144062
epoch time: 1.5800995826721191
epoch 870
training mse: 0.40159458 validation mse 0.40150195
epoch time: 1.562835454940796
epoch 880
training mse: 0.40155497 validation mse 0.4013944
epoch time: 1.5638208389282227
epoch 890
training mse: 0.40150067 validation mse 0.40134615
epoch time: 1.5515491962432861
epoch 900
training mse: 0.40165454 validation mse 0.40154633
epoch time: 1.5598220825